In [1]:
import pandas as pd
from selenium import webdriver
import openpyxl
from selenium.webdriver.common.by import By
import time
def text(texto):
    return float(texto.replace('.','').replace('R$','').replace(',','.'))

In [2]:
products = pd.read_excel(r"C:\Users\Pichau\Desktop\Produtos.xlsx")
products = products.fillna('-')
driver = webdriver.Chrome()
driver.set_window_position(-10000,0)
send_email = False
for i,row in products.iterrows():
    #Amazon Prices
        driver.get(row['Amazon'])
        try:
            price_amazon = driver.find_element(by=By.XPATH,value='//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span/span[2]/span[2]').text
         
        except:
            print('{} not found in Amazon').format(linha['Product Name'])
            price_amazon = linha['Preco Original'] *2
        price_amazon = text(price_amazon)  

    
        #Shopee Prices
        driver.get(row['Shopee'])
        time.sleep(2)
        try:
            time.sleep(2)
            price_shopee =  driver.find_element(by=By.XPATH,value='/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div[2]/div[3]/div/div[3]/div/div/div/div/div/div').text
        except:
            try:
                price_shopee = driver.find_element(by=By.XPATH,value='/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div[2]/div[3]/div/div[3]/div/div/div/div/div/div').text
            except:
                print('{} not found in Shopee').format(linha['Product Name'])
                price_shopee = linha['Preco Original'] *2   
        price_shopee = text(price_shopee)           

        #Magazine Luiza Prices
        driver.get(row['Magazine Luiza'])
        try:
            price_Magazine_Luiza =  driver.find_element(by=By.XPATH,value='/html/body/div[3]/div[5]/div[1]/div[3]/div[2]/div[5]/div/div[2]/div/span[2]').text
        except:
               price_Magazine_Luiza = driver.find_element(by=By.XPATH,value='/html/body/div[3]/div[5]/div[1]/div[3]/div[2]/div[4]/div/div/div/span[2]').text
            
        price_Magazine_Luiza = text(price_Magazine_Luiza)
    
    
        original_price =  row['Original_price']
        list_price = [(price_amazon,'Amazon'),(price_shopee,'Loja Shopee'),(price_Magazine_Luiza,'Magazine Luiza'),(original_price,'Original')]
        list_price.sort()
        products.loc[i,'Current price'] = list_price[0][0]
        products.loc[i,'Local'] = list_price[0][1]
        if list_price[0][0] < original_price * 0.70:
            send_email = True

#Send email
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')
discount = 0.3
if send_email:
    mail = outlook.CreateItem(0)
    mail.To='email'
    mail.Subject=f'Product(s) Found with more than {discount:.0%} Discount'
    #Filter Products:
    table_filtered = products.loc[products['Current price'] <= products['Original_price']*discount,:]
    mail.HTMLBody =f'<p>These are the Products with more than {discount:.0%} Discount</p>{products.to_html()}'
    
    mail.Send()
print('End.')
driver.quit()


End.
